# Test COST values

In [ ]:
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{2025}.csv"
df = pd.read_csv(url, index_col=[0, 1])
df.head()

value           unit  \
technology                       parameter                                    
Alkaline electrolyzer large size FOM                  4.6000         %/year   
                                 VOM                  0.2389     EUR/MWh_H2   
                                 electricity-input    1.3800  MWh_el/MWh_H2   
                                 investment         429.0306         EUR/kW   
                                 lifetime            40.0000          years   

                                                                                               source  \
technology                       parameter                                                              
Alkaline electrolyzer large size FOM                JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 VOM                JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 electricity-input  JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 investment         JRC, 01_JRC-EU-TIMES Full model, https://zenod...   
                                 lifetime           JRC, 01_JRC-EU-TIMES Full model, https://zenod...   

                                                         further description  \
technology                       parameter                                     
Alkaline electrolyzer large size FOM                Reference capacity 72 MW   
                                 VOM                Reference capacity 72 MW   
                                 electricity-input  Reference capacity 72 MW   
                                 investment         Reference capacity 72 MW   
                                 lifetime           Reference capacity 72 MW   

                                                    currency_year  
technology                       parameter                         
Alkaline electrolyzer large size FOM                       2010.0  
                                 VOM                       2010.0  
                                 electricity-input            NaN  
                                 investment                2010.0  
                                 lifetime                     NaN

In [ ]:
investment_units = df[df.index.get_level_values(1).str.contains("investment")]["unit"].unique().tolist()
investment_units

tech_units = {}
for tech in setup_exp[region]:
    # Find all units for this technology in the cost dataframe, only for parameter 'investment'
    units = df.loc[(df.index.get_level_values(0) == tech) & (df.index.get_level_values(1) == "investment"), "unit"].unique().tolist()
    tech_units[tech] = units

tech_units

{'OCGT': ['EUR/kW'],
 'CCGT': ['EUR/kW'],
 'battery storage': ['EUR/kWh'],
 'onwind': ['EUR/kW'],
 'offwind': ['EUR/kW_e, 2020'],
 'solar': ['EUR/kW_e'],
 'electrolysis': ['EUR/kW_e'],
 'fuel cell': ['EUR/kW_e'],
 'Hydrogen storage': [],
 'Reservoir hydro storage': [],
 'Load shedding': []}

In [ ]:
for tech in setup_exp[region]:
    print(f"Investment cost for '{tech}':")
    if tech in tech_units and tech_units[tech]:
        for unit in tech_units[tech]:
            value = df.loc[(tech, "investment"), "value"] if (tech, "investment") in df.index else None
            print(f"  Unit: {unit}, Value: {value}")
    else:
        print("  No investment cost unit found.")

    # Include lifetime parameter
    lifetime = df.loc[(tech, "lifetime"), "value"] if (tech, "lifetime") in df.index else None
    print(f"  Lifetime: {lifetime}")

    # Include FOM parameter
    fom = df.loc[(tech, "FOM"), "value"] if (tech, "FOM") in df.index else None
    print(f"  FOM: {fom}")

Investment cost for 'OCGT':
  Unit: EUR/kW, Value: 470.4853
  Lifetime: 25.0
  FOM: 1.7784
Investment cost for 'CCGT':
  Unit: EUR/kW, Value: 904.7795
  Lifetime: 25.0
  FOM: 3.3392
Investment cost for 'battery storage':
  Unit: EUR/kWh, Value: 198.8558
  Lifetime: 22.5
  FOM: None
Investment cost for 'onwind':
  Unit: EUR/kW, Value: 1139.8826
  Lifetime: 28.5
  FOM: 1.2347
Investment cost for 'offwind':
  Unit: EUR/kW_e, 2020, Value: 1769.1171
  Lifetime: 30.0
  FOM: 2.3741
Investment cost for 'solar':
  Unit: EUR/kW_e, Value: 676.5703
  Lifetime: 37.5
  FOM: 1.7275
Investment cost for 'electrolysis':
  Unit: EUR/kW_e, Value: 1800.0
  Lifetime: 25.0
  FOM: 4.0
Investment cost for 'fuel cell':
  Unit: EUR/kW_e, Value: 1269.866
  Lifetime: 10.0
  FOM: 5.0
Investment cost for 'Hydrogen storage':
  No investment cost unit found.
  Lifetime: None
  FOM: None
Investment cost for 'Reservoir hydro storage':
  No investment cost unit found.
  Lifetime: None
  FOM: None
Investment cost for 'Loa

In [ ]:
for tech, active in setup[region].items():
    if active and tech in Cost.costs.index:
        cap_cost = Cost.costs.loc[tech, "capital_cost"]
        unit = Cost.units.get((tech, "investment"), "")
        print(f"{tech}: {cap_cost:.2f} EUR/MW | Unit: {unit}")

# Also look up battery inverter and battery storage
if "battery inverter" in Cost.costs.index:
    cap_cost_battery_inv = Cost.costs.loc["battery inverter", "capital_cost"]
    unit_battery_inv = Cost.units.get(("battery inverter", "investment"), "")
    print(f"battery inverter: {cap_cost_battery_inv:.2f} EUR/MW | Unit: {unit_battery_inv}")



OCGT: 48739.70 EUR/MW | Unit: EUR/MW
battery storage: 17805.06 EUR/MW | Unit: EUR/MWh
onwind: 107441.46 EUR/MW | Unit: EUR/MW
solar: 63114.92 EUR/MW | Unit: EUR/MW_e
electrolysis: 226458.93 EUR/MW | Unit: EUR/MW_e
fuel cell: 244293.65 EUR/MW | Unit: EUR/MW_e
battery inverter: 33126.23 EUR/MW | Unit: EUR/MW


In [ ]:
# JUST THE PRINT
def print_costs(network):
    gens = network.generators[["carrier", "capital_cost", "marginal_cost"]].copy()
    gens["type"] = "generator"

    sus = network.storage_units[["carrier", "capital_cost", "marginal_cost"]].copy()
    sus["type"] = "storage"

    df = pd.concat([gens, sus])
    df.reset_index(inplace=True)
    df.rename(columns={"index": "name"}, inplace=True)

    print(df[["name", "type", "carrier", "capital_cost", "marginal_cost"]])
print(" COST FOR PERFECT FORESIGHT NETWORK ")

In [ ]:
# WITH DF 
def print_costs(network):
    # Generators
    gens = network.generators[["carrier", "capital_cost", "marginal_cost"]].copy()
    gens["type"] = "generator"

    # Storage Units
    sus = network.storage_units[["carrier", "capital_cost", "marginal_cost"]].copy()
    sus["type"] = "storage"

    # Combine
    df = pd.concat([gens, sus])
    df.reset_index(inplace=True)
    df.rename(columns={"index": "name"}, inplace=True)

    # Print nicely
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 120)
    pd.set_option("display.float_format", "{:,.2f}".format)

    print(df[["name", "type", "carrier", "capital_cost", "marginal_cost"]])

    return df